# Transformer
RNN을 제거하고, Attention으로만 구성된 서로 다른 인코더와 디코더가 여러 개 쌓인 형태의 네트워크

인코더 또는 디코더 안에는 self attention과 feedford 신경망(Dense)으로 구성 되어 있다. - 대다수의 lmm의 구조는 거의 대부분 transform으로 구성되어있다.

self attention함수는 주어진 Query에 대해서 key와의 유사도를 구한다. 그리고 이 유사도를 키와 매핑되어 있는 값(value)에 반영한다.

self attention은 RNN 처럼 순서대로 처리하는 방식이 아니라, 해당하는 단어와 관련된 뜻을 찾기 위한 어텐션을 말한다.

In [ ]:
# 간단한 구조 이해

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential

In [ ]:
# load data
vocab_size =20000
maxlen=200

(x_train,y_train),(x_val,y_val)= keras.datasets.imdb.load_data(num_words=vocab_size)
x_val= keras.preprocessing.sequence.pad_sequences(x_val,maxlen=maxlen)
print(x_val)

In [ ]:
# Token and Position embedding
inputs=layers.Input(shape=(200,))
x=layers.Embedding(input_dim=128,output_dim=32)(inputs)
print(tf.keras.Model(inputs=inputs,outputs=x).summary())


In [ ]:
inputs=layers.Input(shape=(200,))
x=layers.Embedding(input_dim=128,output_dim=32)(inputs)
positions=tf.range(start=0,limit=200, delta=1) # 각 단어의 위치에 따른 index값을 embedding하여 feature를 추출한 후, 단어 embeding 결과 + 위치 embedding결과의 합을 구하는
positions=layers.Embedding(input_dim=200,output_dim=32)(positions)
embedding_layer =x+positions

# Multi-head attention을 위한 간단한 구조를 작성
EmbeddingDim=5
WordLen=5

inputs= layers.Input(shape=(WordLen,))
positions=tf.range(start=0,limit=WordLen, delta=1)
positions=layers.Embedding(input_dim=WordLen,output_dim=EmbeddingDim)(positions)
x=layers.Embedding(input_dim=200,output_dim=EmbeddingDim)(inputs)
embedding_layer=x+positions

#num_heads=1 : 현재 Attention Matrix를 만드는 작업을 몇번할 것인가를 결정
attention_output=layers.MultiHeadAttention(num_heads=1, key_dim=1, use_bias=False)(embedding_layer, embedding_layer)
print(tf.keras.Model(inputs=inputs,outputs=x).summary())

model=tf.keras.Model(inputs=inputs,outputs=attention_output)
print([w.shape for w in model.get_weights()])
# [(200, 5), (5, 1, 1), (5, 1, 1), (5, 1, 1), (1, 1, 5)]
# embedding_layer weight, Query, Key, Value 에 대한 weight dot production내부에서 transpose되어 순서가 변경

print('--------------------------')
# MultiheadAttention layer에 num_heads, key_dim=3으로 변경해보자
inputs= layers.Input(shape=(WordLen,))
positions=tf.range(start=0,limit=WordLen, delta=1)
positions=layers.Embedding(input_dim=WordLen,output_dim=EmbeddingDim)(positions)
x=layers.Embedding(input_dim=200,output_dim=EmbeddingDim)(inputs)
embedding_layer=x+positions

# num_heads=1 : 현재 Attention Matrix를 만드는 작업을 몇번할 것인가를 결정
# attention_output=layers.MultiHeadAttention(num_heads=1, key_dim=1, use_bias=False)(embedding_layer, embedding_layer)
attention_output=layers.MultiHeadAttention(num_heads=1, key_dim=1, use_bias=True)(embedding_layer, embedding_layer)
print(tf.keras.Model(inputs=inputs,outputs=x).summary())

model=tf.keras.Model(inputs=inputs,outputs=attention_output)
print([w.shape for w in model.get_weights()])
# [(200, 5), (5, 1, 1), (5, 1, 1), (5, 1, 1), (1, 1, 5)]
# [(200, 5), (5, 1, 1), (1, 1), (5, 1, 1), (1, 1), (5, 1, 1), (1, 1), (1, 1, 5), (5,)] use bias =True하면 중간에 상수가 더해짐
# 파라미터에 대한 존재 이유, 연산 등의 이해가 있다면, 다음으로 Scaled dot production attention만 이해하면 Transformer에 기초 끝
# 단어들의 유사도와 가튼 역할을 하는 Attention score를 구하고 이를


In [ ]:
# Multi-head attention를 통과한 분류 모델을 작성하기
EmbeddingDim=5
WordLen=5
inputs= layers.Input(shape=(WordLen,))
positions=tf.range(start=0,limit=WordLen, delta=1)
positions=layers.Embedding(input_dim=WordLen,output_dim=EmbeddingDim)(positions)
x=layers.Embedding(input_dim=200,output_dim=EmbeddingDim)(inputs)
embedding_layer=x+positions

# attention_output=layers.MultiHeadAttention(num_heads=1, key_dim=1, use_bias=False)(embedding_layer, embedding_layer)
attention_output=layers.MultiHeadAttention(num_heads=1, key_dim=1, use_bias=True)(embedding_layer, embedding_layer)

x=layers.GlobalAveragePooling1D()(attention_output) #MultiHeaAttention을 통과한 output은 Embedding Layer의 output과 동일하므로, GlobalAveragePooling1D를 통해 vector화 할수 있다.
outputs=layers.Dense(2, activation='softmax')(x)

model=tf.keras.Model(inputs=inputs,outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc'])
print(model.summary())